In [94]:
import requests
import bs4
import pprint as pp
import re

In [22]:
site = requests.get("https://arxiv.org/list/astro-ph.CO/recent")

# Goal: get names of articles, authors and download pdf


In [152]:
site.text[:1000]

'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en">\n<head>\n<title>Cosmology and Nongalactic Astrophysics  authors/titles recent submissions</title>\n<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" />\n<link rel="stylesheet" type="text/css" media="screen" href="/css/arXiv.css?v=20190306" />\n<link rel="stylesheet" type="text/css" media="screen" href="/bibex/bibex.css?20181009">\n<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"> \n<link rel="alternate" type="application/rss+xml" title="Cosmology and Nongalactic Astrophysics " href="http://arxiv.org/rss/astro-ph.CO"/>\n<script src="/js/mathjaxToggle.min.js" type="text/javascript"></script>\n\n<!-- Piwik -->\n<script type="text/javascript">\nvar _paq = _paq || [];\n_paq.push(["setDomains", ["*.arxiv.o

In [24]:
parser = bs4.BeautifulSoup(site.text, 'lxml')

In [156]:
print(parser)

<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Cosmology and Nongalactic Astrophysics  authors/titles recent submissions</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/css/arXiv.css?v=20190306" media="screen" rel="stylesheet" type="text/css"/>
<link href="/bibex/bibex.css?20181009" media="screen" rel="stylesheet" type="text/css"/>
<link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet"/>
<link href="http://arxiv.org/rss/astro-ph.CO" rel="alternate" title="Cosmology and Nongalactic Astrophysics " type="application/rss+xml"/>
<script src="/js/mathjaxToggle.min.js" type="text/javascript"></script>
<!-- Piwik -->
<script type="text/javascript">
var _paq = _paq || [];
_paq.push(["setDomains", ["*.arxiv.org"]]);
_paq.push([

In [115]:
# span class="descriptor"
"""
cntr + shift + I => Inspector in any browser(chore, firefox, etc)


<div class="list-title mathjax">
<span class="descriptor">Title:</span> Joining bits and pieces of reionization history
</div>

<span class="list-identifier"><a href="/abs/1904.02129" title=
"Abstract">arXiv:1904.02129</a> [<a href="/pdf/1904.02129" title="Download PDF">pdf</a>, 
<a href="/ps/1904.02129" title="Download PostScript">ps</a>,
<a href="/format/1904.02129" title="Other formats">other</a>]</span>

"""

articles_list = parser.find_all('div', attrs={'class':'list-title mathjax'})
authors_list = parser.find_all('div', attrs={'class': 'list-authors'})
print(len(articles_list))
print(len(authors_list))

# Ok, seems like now we can zip(list,list) and combine them, still we have to clear from html stuff 

25
25


In [147]:
# tired of searching bs4 docs >_<

pattern = re.compile(">(.+)<\/a>") 
rm_auth_announce = re.compile('(Authors:)+')

authors_cleared = [rm_auth_announce.sub("", i.text).replace('\n', "").split(',') 
                   for i in authors_list]

In [150]:
article_name = re.compile("(?<=Title:<\/span>).+(?<!<\/div.)")
rm_title_announce = re.compile("(Title:)+")

articles_cleared = [rm_title_announce.sub("", i.text).replace('\n', "") 
                    for i in articles_list]

In [148]:
print(authors_cleared[:1])
pp.pprint(articles_cleared[:1])

[['Bernard Carr', ' Sebastien Clesse', ' Juan García-Bellido']]
[' Primordial black holes, dark matter and hot-spot electroweak  baryogenesis '
 'at the quark-hadron epoch']


In [149]:
articles_with_authors = list(zip(articles_cleared,authors_cleared))
pp.pprint(articles_with_authors[0])

(' Primordial black holes, dark matter and hot-spot electroweak  baryogenesis '
 'at the quark-hadron epoch',
 ['Bernard Carr', ' Sebastien Clesse', ' Juan García-Bellido'])


In [ ]:
# pdf download to be continuied in scrapy section